#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import math

#### Utility

In [2]:
epsilon = 1e-6
class BasicUtil:
    
    @staticmethod
    def roundUp(x) -> int:
        return math.ceil(x)
    
    @staticmethod
    def roundDown(x) -> int:
        return math.floor(x)

    @staticmethod
    def isClose(x,y) -> bool:
        return abs(x - y) <= epsilon
    

#### Change .txt to .csv file

In [3]:
def change(name):
    f = open(f"{name}.txt", "r")

    out = open(f"{name}.csv", "w")

    for line in f:
        change = line.replace("\t", ",")
        out.write(change)

In [4]:
change("Facility")
change("Demand")
change("TransCost")

#### Read data from csv files

In [5]:
def read_file(filename):
    with open(filename,"r") as f:
        df = pd.read_csv(f,header=None)
        return df.to_numpy()

In [6]:
facilities = read_file("Facility.csv")
shops = read_file("Demand.csv")
transport = read_file("TransCost.csv")

In [7]:
# print(facilities)
# print(demand)

#### Create and Optimize model

In [8]:
# ---------------------------------------------------
# Create model
Env = gp.Env()
# Env.setParam('OutputFlag', 0)
model = gp.Model("2.2.1", env = Env)
model.setParam(GRB.Param.PoolSolutions,9)
model.setParam(GRB.Param.PoolSearchMode,2)
# ---------------------------------------------------
# Define variables
nFactories = 10
nShops = 200
MAX_CAPACITY = facilities[:,3]
x = model.addMVar(shape = (nFactories, nShops), lb = 0, ub = GRB.INFINITY, vtype = GRB.INTEGER, name = "x")
chose = model.addMVar(shape = nFactories, vtype = GRB.BINARY, name = "chose")     
demand = shops[:,3]
build = facilities[:,-1]
model.update()

# ---------------------------------------------------
# Define constraints
model.addConstr(x.sum(axis = 1) <= MAX_CAPACITY * chose)
model.addConstr(x.sum(axis = 0) >= demand, name = "shop-demand")

# ---------------------------------------------------
# Define objective
model.setObjective((x * transport).sum() + (chose * build).sum(), GRB.MINIMIZE)
# model.setParam("PoolSolutions", 10)
# model.setParam("PoolSearchMode", 2)


model.optimize()

print(model.Fingerprint)
print(model.Runtime)
print(model.NumConstrs)
print(model.NumVars)
print(model.SolCount)
print(model.ObjVal)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2395694
Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Set parameter PoolSolutions to value 9
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Optimize a model with 210 rows, 2010 columns and 4010 nonzeros
Model fingerprint: 0x601840fa
Variable types: 0 continuous, 2010 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e-01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 4e+01]
Found heuristic solution: objective 23918.380000
Presolve time: 0.00s
Presolved: 210 rows, 2010 columns

In [9]:
model.write("2.2.1.mps")
model.write("2.2.1.lp")

In [10]:
output_main = None
test = x.Xn
if model.status == GRB.OPTIMAL:
    print("One solution found")
    print(chose.X)
    # with open("sol-main.txt","w") as f:
    #     for i in range(nFactories):
    #         for j in range(nShops):
    #             f.write(f"{abs(x[i,j].X)} ")
    #         f.write("\n")
    output_main = x.X
    print(x.Xn)
    # print(x.X.sum())
    for i in range(nFactories):
        if chose[i].X > 0:
            print('Factory ', i, ' is chosen')
            print(x[i].X.sum())
            for j in range(nShops):
                if x[i,j].x > 0:
                    print(f"Factory {i} sells {x[i][j].X} to shop {j}")
    print(model.ObjVal)


else:
    print('No optimal solution found')

One solution found
[0. 1. 0. 1. 0. 1. 0. 1. 1. 0.]
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  7. 16.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0. 17.]
 [ 0.  0.  0. ...  0.  0.  0.]]
Factory  1  is chosen
1000.0
Factory 1 sells 9.0 to shop 10
Factory 1 sells 12.0 to shop 12
Factory 1 sells 9.0 to shop 58
Factory 1 sells 34.0 to shop 79
Factory 1 sells 32.0 to shop 97
Factory 1 sells 44.0 to shop 104
Factory 1 sells 39.0 to shop 105
Factory 1 sells 29.0 to shop 107
Factory 1 sells 25.0 to shop 108
Factory 1 sells 28.0 to shop 111
Factory 1 sells 38.0 to shop 112
Factory 1 sells 9.0 to shop 113
Factory 1 sells 29.0 to shop 114
Factory 1 sells 31.0 to shop 118
Factory 1 sells 39.0 to shop 119
Factory 1 sells 7.0 to shop 120
Factory 1 sells 20.0 to shop 124
Factory 1 sells 16.0 to shop 125
Factory 1 sells 9.0 to shop 126
Factory 1 sells 13.0 to shop 127
Factory 1 sells 26.0 to shop 129
Factory 1 sells 28.0 to shop 130
Factory 1 se

In [11]:
# for _ in range(model.SolCount):
#     model.setParam(GRB.Param.SolutionNumber,_)
#     print("One solution found")
#     with open(f"sol-main-{_}.txt","w") as f:
#         f.write(f"Solution objective: {model.PoolObjVal}\n")
#         f.write(f"Factory chosen: ")
#         for i in range(nFactories):
#             if chose[i].Xn == 1:
#                 f.write(f"{i} ")
#         f.write("\n")
#         for i in range(nFactories):
#             for j in range(nShops):
#                 f.write(f"{abs(x[i,j].Xn)} ")
#             f.write("\n")
#     output_main = x.Xn
#     print(x.Xn)
#     print(model.PoolObjVal)
#     # print(x.X.sum())
#     for i in range(nFactories):
#         if chose[i].X > 0:
#             print('Factory ', i, ' is chosen')
#             print(x[i].X.sum())
#             for j in range(nShops):
#                 if x[i,j].x > 0:
#                     print(f"Factory {i} sells {x[i][j].X} to shop {j}")
#     print(model.ObjVal)

#### Prove that the solution above is the only optimal solution

Choosing different set of facility

In [12]:
cnst = 0
cnt_facility = 0
for i in range(nFactories):
    if (chose[i].X == 1):
        cnst = cnst + chose[i]
        cnt_facility += 1

model.addConstr(cnst <= cnt_facility-1, name="diff-facility")

model.update()

model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Optimize a model with 211 rows, 2010 columns and 4015 nonzeros
Model fingerprint: 0x72e53450
Variable types: 0 continuous, 2010 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e-01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+01]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint diff-facility by 1.000000000

Found heuristic solution: objective 22767.800000
Presolve time: 0.00s
Presolved: 211 rows, 2010 columns, 4015 nonzeros
Variable types: 0 continuous, 2010 integer (10 binary)
Found heuristic solution: objectiv

Root relaxation: objective 1.093869e+04, 328 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 10938.6900    0    6 13669.0400 10938.6900  20.0%     -    0s
H    0     0                    12846.840000 10938.6900  14.9%     -    0s
H    0     0                    12846.520000 10938.6900  14.9%     -    0s
H    0     0                    12845.420000 10938.6900  14.8%     -    0s
     0     0 11030.9000    0   31 12845.4200 11030.9000  14.1%     -    0s
H    0     0                    12844.540000 11118.1708  13.4%     -    0s
H    0     0                    12383.430000 11118.1708  10.2%     -    0s
     0     0 11196.5100    0   27 12383.4300 11196.5100  9.58%     -    0s
H    0     0                    12189.540000 11196.5100  8.15%     -    0s
     0     0 11303.9200    0   45 12189.5400 11303.9200  7.27%     -    0s
     0    

In [13]:
output_diff_fac = None
if model.status == GRB.OPTIMAL:
    print("One solution found")
    # with open("sol-diff-facility.txt","w") as f:
    #     for i in range(nFactories):
    #         for j in range(nShops):
    #             f.write(f"{abs(x[i,j].X)} ")
    #         f.write("\n")
    output_diff_fac = x.X
    for i in range(nFactories):
        if chose[i].X > 0:
            print('Factory ', i, ' is chosen')
            print(x[i].X.sum())
            for j in range(nShops):
                if x[i,j].x > 0:
                    print(f"Factory {i} sells {x[i][j].X} to shop {j}")
    print(model.ObjVal)


else:
    print('No optimal solution found')

One solution found
Factory  2  is chosen
1000.0
Factory 2 sells 9.0 to shop 10
Factory 2 sells 12.0 to shop 12
Factory 2 sells 7.0 to shop 20
Factory 2 sells 7.0 to shop 54
Factory 2 sells 13.0 to shop 58
Factory 2 sells 29.0 to shop 65
Factory 2 sells 21.0 to shop 69
Factory 2 sells 34.0 to shop 79
Factory 2 sells 32.0 to shop 97
Factory 2 sells 35.0 to shop 99
Factory 2 sells 25.0 to shop 102
Factory 2 sells 39.0 to shop 105
Factory 2 sells 29.0 to shop 107
Factory 2 sells 25.0 to shop 108
Factory 2 sells 28.0 to shop 111
Factory 2 sells 38.0 to shop 112
Factory 2 sells 9.0 to shop 113
Factory 2 sells 31.0 to shop 118
Factory 2 sells 39.0 to shop 119
Factory 2 sells 7.0 to shop 120
Factory 2 sells 20.0 to shop 124
Factory 2 sells 16.0 to shop 125
Factory 2 sells 9.0 to shop 126
Factory 2 sells 13.0 to shop 127
Factory 2 sells 26.0 to shop 129
Factory 2 sells 16.0 to shop 139
Factory 2 sells 7.0 to shop 142
Factory 2 sells 17.0 to shop 143
Factory 2 sells 12.0 to shop 144
Factory 2 se

Different facility output

In [14]:
model.remove(model.getConstrByName("diff-facility"))

cnst = 0
total_output = 0
for i in range(nFactories):
    for j in range(nShops):
        if output_main[i,j] > 0:
            cnst += x[i,j]
            total_output += x[i,j].X

# print(total_output)
print(cnst)

model.addConstr(cnst <= total_output-1, name="diff-output")

model.update()

model.optimize()

<MLinExpr ()   >
array( x[1,10] + x[1,12] + x[1,58] + x[1,79] + x[1,97] + x[1,104] + x[1,105] + x[1,107] + x[1,108] + x[1,111] + x[1,112] + x[1,113] + x[1,114] + x[1,118] + x[1,119] + x[1,120] + x[1,124] + x[1,125] + x[1,126] + x[1,127] + x[1,129] + x[1,130] + x[1,139] + x[1,140] + x[1,142] + x[1,143] + x[1,144] + x[1,145] + x[1,150] + x[1,157] + x[1,159] + x[1,160] + x[1,163] + x[1,165] + x[1,166] + x[1,168] + x[1,170] + x[1,172] + x[1,179] + x[1,186] + x[1,195] + x[1,197] + x[1,198] + x[3,0] + x[3,5] + x[3,6] + x[3,11] + x[3,14] + x[3,18] + x[3,19] + x[3,21] + x[3,29] + x[3,33] + x[3,36] + x[3,39] + x[3,40] + x[3,42] + x[3,43] + x[3,44] + x[3,45] + x[3,46] + x[3,52] + x[3,53] + x[3,54] + x[3,57] + x[3,58] + x[3,63] + x[3,65] + x[3,69] + x[3,73] + x[3,75] + x[3,77] + x[3,81] + x[3,84] + x[3,85] + x[3,87] + x[3,92] + x[3,94] + x[3,96] + x[3,99] + x[3,123] + x[3,138] + x[3,152] + x[3,187] + x[3,194] + x[5,1] + x[5,2] + x[5,3] + x[5,4] + x[5,7] + x[5,8] + x[5,9] + x[5,13] + x[5,15] + x[5

In [15]:
output_diff_out = None
if model.status == GRB.OPTIMAL:
    print("One solution found")
    # with open("sol-diff-output.txt","w") as f:
    #     for i in range(nFactories):
    #         for j in range(nShops):
    #             f.write(f"{abs(x[i,j].X)} ")
    #         f.write("\n")
    output_diff_out = x.X
    for i in range(nFactories):
        if chose[i].X > 0:
            print('Factory ', i, ' is chosen')
            print(x[i].X.sum())
            for j in range(nShops):
                if x[i,j].x > 0:
                    print(f"Factory {i} sells {x[i][j].X} to shop {j}")
    print(model.ObjVal)


else:
    print('No optimal solution found')

One solution found
Factory  2  is chosen
1000.0
Factory 2 sells 9.0 to shop 10
Factory 2 sells 12.0 to shop 12
Factory 2 sells 7.0 to shop 20
Factory 2 sells 7.0 to shop 54
Factory 2 sells 13.0 to shop 58
Factory 2 sells 29.0 to shop 65
Factory 2 sells 22.0 to shop 69
Factory 2 sells 34.0 to shop 79
Factory 2 sells 32.0 to shop 97
Factory 2 sells 35.0 to shop 99
Factory 2 sells 25.0 to shop 102
Factory 2 sells 39.0 to shop 105
Factory 2 sells 29.0 to shop 107
Factory 2 sells 25.0 to shop 108
Factory 2 sells 28.0 to shop 111
Factory 2 sells 38.0 to shop 112
Factory 2 sells 9.0 to shop 113
Factory 2 sells 31.0 to shop 118
Factory 2 sells 39.0 to shop 119
Factory 2 sells 7.0 to shop 120
Factory 2 sells 20.0 to shop 124
Factory 2 sells 16.0 to shop 125
Factory 2 sells 9.0 to shop 126
Factory 2 sells 13.0 to shop 127
Factory 2 sells 26.0 to shop 129
Factory 2 sells 16.0 to shop 139
Factory 2 sells 7.0 to shop 142
Factory 2 sells 17.0 to shop 143
Factory 2 sells 11.0 to shop 144
Factory 2 se

#### Checking solution

Different facilities

In [16]:
# print(output_diff_out)
# print(output_diff_fac)
# print(output_main)
for i in range(nFactories):
    for j in range(nShops):
        if (output_main[i,j] != output_diff_out[i,j]):
            print(f"Diff out: Facility {i} Shop {j}: main = {output_main[i,j]}, diff_out = {output_diff_out[i,j]}")

for i in range(nFactories):
    for j in range(nShops):
        if (output_main[i,j] != output_diff_fac[i,j]):
            print(f"Diff fac: Facility {i} Shop {j}: main = {output_main[i,j]}, diff_out = {output_diff_fac[i,j]}")

Diff out: Facility 1 Shop 10: main = 9.0, diff_out = 0.0
Diff out: Facility 1 Shop 12: main = 12.0, diff_out = 0.0
Diff out: Facility 1 Shop 58: main = 9.0, diff_out = 0.0
Diff out: Facility 1 Shop 79: main = 34.0, diff_out = 0.0
Diff out: Facility 1 Shop 97: main = 32.0, diff_out = 0.0
Diff out: Facility 1 Shop 104: main = 44.0, diff_out = 0.0
Diff out: Facility 1 Shop 105: main = 39.0, diff_out = 0.0
Diff out: Facility 1 Shop 107: main = 29.0, diff_out = 0.0
Diff out: Facility 1 Shop 108: main = 25.0, diff_out = 0.0
Diff out: Facility 1 Shop 111: main = 28.0, diff_out = 0.0
Diff out: Facility 1 Shop 112: main = 38.0, diff_out = 0.0
Diff out: Facility 1 Shop 113: main = 9.0, diff_out = 0.0
Diff out: Facility 1 Shop 114: main = 29.0, diff_out = 0.0
Diff out: Facility 1 Shop 118: main = 31.0, diff_out = 0.0
Diff out: Facility 1 Shop 119: main = 39.0, diff_out = 0.0
Diff out: Facility 1 Shop 120: main = 7.0, diff_out = 0.0
Diff out: Facility 1 Shop 124: main = 20.0, diff_out = 0.0
Diff o

#### Export model

In [17]:
# model.write("2.2.1.mps")